In [1]:
# Importing Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from numpy import linalg as LA

In [2]:
# Loading Source & Target Dataset
X = np.load("Data/olivetti_faces.npy")
Y = np.load("Data/olivetti_faces_target.npy")

In [3]:
# Splitting into Train & Test Dataset
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [4]:
X_train.shape

(300, 64, 64)

In [5]:
X_test.shape

(100, 64, 64)

In [6]:
# Flattening & Reshaping Data
X_train = X_train.flatten().reshape(300,4096)
X_test = X_test.flatten().reshape(100,4096)

In [7]:
# Standardizing Training Data
scaler1 = skp.StandardScaler(copy=False).fit(X_train)
scaler1.transform(X_train)

array([[-0.95935017, -0.5766609 , -0.40062758, ...,  0.92574406,
         0.9085854 ,  0.7555955 ],
       [-1.6723652 , -1.9348845 , -0.78612095, ...,  0.90439725,
         1.1295806 ,  1.3207661 ],
       [ 0.7886866 ,  0.82537645,  0.8629342 , ..., -1.1875995 ,
        -1.102472  , -1.0981644 ],
       ...,
       [ 0.28267604,  1.2635132 ,  1.6339209 , ..., -1.0381713 ,
        -1.0361733 , -1.030344  ],
       [-1.0513521 , -1.1900522 , -1.4500262 , ...,  1.1392133 ,
         0.22349997, -0.5103869 ],
       [ 0.2596755 ,  0.19007836, -0.01513426, ...,  1.651539  ,
         1.7041686 ,  0.8912365 ]], dtype=float32)

In [8]:
# Standardizing Test Data
scaler2 = skp.StandardScaler(copy=False).fit(X_test)
scaler2.transform(X_test)

array([[ 1.1705022 ,  1.096167  ,  0.7910445 , ..., -1.2601049 ,
        -1.2511982 , -1.2851127 ],
       [ 1.5094512 ,  1.2260702 ,  0.99877656, ..., -0.44840354,
        -0.40823504, -0.49670002],
       [-0.88578564, -0.39772037,  0.04320839, ..., -0.6697766 ,
        -0.60091233, -0.6400478 ],
       ...,
       [-0.6372234 , -1.0905375 , -1.2447313 , ..., -1.0879259 ,
        -0.8899283 , -0.37724352],
       [ 0.2892361 , -0.07296232,  0.27171376, ...,  0.58467096,
         0.4828974 ,  0.45895174],
       [ 0.78636074,  0.59820455,  0.5417655 , ...,  0.11732771,
        -0.986267  , -0.71172166]], dtype=float32)

In [9]:
# Calculating Covariance of Train Data
# Take Transpose of Training Data before calculating covariance
# Bcoz we want variability between features i.e. between columns not between rows
X_train_cov = np.cov(X_train.T)

In [10]:
X_train_cov.shape

(4096, 4096)

In [11]:
X_train_cov

array([[ 1.0033444 ,  0.94744189,  0.8164271 , ..., -0.16278   ,
        -0.18061455, -0.14454212],
       [ 0.94744189,  1.00334382,  0.92200701, ..., -0.24200364,
        -0.25745395, -0.21128855],
       [ 0.8164271 ,  0.92200701,  1.00334427, ..., -0.34298165,
        -0.34057894, -0.2907474 ],
       ...,
       [-0.16278   , -0.24200364, -0.34298165, ...,  1.00334438,
         0.92101021,  0.83811675],
       [-0.18061455, -0.25745395, -0.34057894, ...,  0.92101021,
         1.00334467,  0.9471965 ],
       [-0.14454212, -0.21128855, -0.2907474 , ...,  0.83811675,
         0.9471965 ,  1.00334455]])

In [12]:
# Calculating Eigen Values & it's corresponding Eigen Vectors
eig_values, eig_vectors = LA.eig(X_train_cov)

In [17]:
eig_values[0]

(1122.8981483742148+0j)

In [18]:
eig_vectors[:,0]

array([-0.00408696+0.j, -0.0054459 +0.j, -0.00697279+0.j, ...,
        0.00152933+0.j,  0.00385955+0.j,  0.00323643+0.j])

In [15]:
eig_values.shape

(4096,)

In [16]:
eig_vectors.shape

(4096, 4096)

In [28]:
# Make a Tuple as (Eigen Value, Eigen Vector) & Sort it
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:,i]) for i in range(len(eig_values))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)

In [29]:
eig_pairs[0][0]

1122.8981483742148

In [30]:
# Define a Threshold Upon Which We Can Decide Whether To Keep Or Discard Which Feature Vector.
exp_var_percentage = 90.0

tot = sum(eig_values)
var_exp = [(i / tot)*100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

num_vec_to_keep = 0

# Simply Count The Number Of Feature Vectors Which We Would Like To Keep Based On A Selected Threshold of 90%.
for index, percentage in enumerate(cum_var_exp):
    if percentage > exp_var_percentage:
        num_vec_to_keep = index + 1
        break

In [31]:
num_vec_to_keep

61

In [23]:
# Project Our Data Onto The Vectors We Decided To Keep. We Do This By Building A Projection Matrix.
num_features = X_train.shape[1]
proj_mat = eig_pairs[0][1].reshape(num_features,1)
for eig_vec_idx in range(1, num_vec_to_keep):
    proj_mat = np.hstack((proj_mat, eig_pairs[eig_vec_idx][1].reshape(num_features,1)))

In [24]:
proj_mat.shape

(4096, 61)

In [32]:
# Recast The Data Along The Principal Components Axes
X_train_PCA = X_train @ (proj_mat)
X_test_PCA = X_test @ (proj_mat)

In [33]:
X_train_PCA.shape

(300, 61)

In [34]:
# Selecting Only Real Part of Eigen Vectors
X_train_PCA = np.real(X_train_PCA)
X_test_PCA = np.real(X_test_PCA)

In [35]:
# Finally Applying Any Of The Classification Methods For Classifying The Data
classifier = LogisticRegression(solver = 'liblinear', multi_class = 'ovr')
classifier.fit(X_train_PCA, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [36]:
# Making Predictions
Y_pred = classifier.predict(X_test_PCA)

In [37]:
# Printing Classification Report
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         4
           3       0.67      1.00      0.80         2
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         4
          12       1.00      0.80      0.89         5
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         4
          17       1.00      1.00      1.00         1
          18       1.00      1.00      1.00         2
          19       1.00    

In [38]:
# Printing Confusion Matrix
print(confusion_matrix(Y_test, Y_pred))

[[4 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 4 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 2]]


In [39]:
# Printing Accuracy
print(accuracy_score(Y_test, Y_pred))

0.98
